In [1]:
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
import re
import matplotlib.pyplot as plt
import transformers
import datasets
import glob

import torch
import json
from pathlib import Path
from datasets import load_dataset, Dataset, load_metric
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, default_data_collator,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          TrainingArguments, Trainer)
from transformers.modeling_outputs import SequenceClassifierOutput

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, precision_score, recall_score

from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler
from tabulate import tabulate


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
pd.set_option('display.max_colwidth', None)

In [3]:
import torch 
import random
torch.manual_seed(123)
torch.cuda.manual_seed(123)
np.random.seed(123)
random.seed(123)
ros = RandomOverSampler(random_state=123)

In [4]:
def load_ds(ds_path):
    df = pd.read_pickle(ds_path)

    #Aligning the 5-levels labels to eli5 ones
        
    #'(D06) To answer - Other' -> '(D06) Answer - Other'
    #'(D07) To provide agreement statement' -> '(D07) Agreement'
    #'(D08) To provide disagreement statement' -> '(D08) Disagreement'
    #'(D10) Other' -> '(D09) Other'
    #'(D09) To provide informing statement' -> (D10) To provide informing statement
    
    
    # (E10) Other -> (E09) Other 
    # (E09) Introducing Extraneous Information -> (E10) Introducing Extraneous Information
    
    df['exp_act_label'] = df.exp_act_label.apply(lambda x: '(E10) Other' if x == '(E09) Other' else x)
    df['exp_act_label'] = df.exp_act_label.apply(lambda x: '(E09) Introducing Extraneous Information' if x == '(E10) Introducing Extraneous Information' else x)

    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D09) Other' if x == '(D10) Other' else x)
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D10) To provide informing statement' if x == '(D09) To provide informing statement' else x)
    
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D06) Answer - Other' if x == '(D06) To answer - Other' else x)
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D07) Agreement' if x == '(D07) To provide agreement statement' else x)
    df['dlg_act_label'] = df.dlg_act_label.apply(lambda x: '(D08) Disagreement' if x == '(D08) To provide disagreement statement' else x)
    

    df['turn_text_with_topic'] = df.apply(lambda row: {
                                        'author': row['turn_text']['author'], 
                                        'text'  : row['topic'].replace('_', ' ') + ' [SEP] ' +  row['turn_text']['text']
                                       } ,axis=1)

    return df

In [5]:
fivelvls_annotation_df = load_ds('../../data/five_levels_ds/annotation-results/MACE-measure/final_mace_predictions.pkl')
eli5_annotation_df     = load_ds('../../data/eli5_ds/annotation-results/MACE-measure/final_mace_predictions_training.pkl')

In [6]:
fivelvls_annotation_df['ds'] = ['5lvls'] * len(fivelvls_annotation_df)
eli5_annotation_df['ds'] = ['eli5'] * len(eli5_annotation_df)
dlgs_df = pd.concat([fivelvls_annotation_df, eli5_annotation_df])

In [7]:
dlgs_df['ds'].value_counts()

eli5     2799
5lvls    1550
Name: ds, dtype: int64

In [8]:
dlgs_df.head()

,task_id,turn_id,topic_func_label,dlg_act_label,exp_act_label,topic,lvl,turn_text,turn_text_with_topic,ds,exp_type_label,post_link,post_id,turn_text_only
0,142,1,(T04) Other - No topic was introduced,(D10) To provide informing statement,(E02) Testing prior knowledge,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'Mostly today, I've been talking a lot about what can we do, what's possible. We think might be possible in the next couple of years. But really at the professional level its more the question of wisdom of what should we be doing.'}","{'author': 'Explainer', 'text': 'virtual reality [SEP] Mostly today, I've been talking a lot about what can we do, what's possible. We think might be possible in the next couple of years. But really at the professional level its more the question of wisdom of what should we be doing.'}",5lvls,NaN,NaN,NaN,NaN
1,142,2,(T01) It is the main topic,(D10) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainee', 'text': 'That's one of the things we're trying to figure out, from an artist and storytelling perspective, what are the things that will make this meaningfully different from what we're used to, like a television on our wall. And we've been finding a lot of things, aspects of virtual reality that very much do that in my opinion. Things that allow you to feel presence, first and foremost, where you get lost, and you have to remind yourself, this isn't actually happening. And things that ultimately allow you to embody other characters. Things where you can actually change your own self-perception and play with neuro-plasticity and teach yourself things that are bizarre and unique.'}","{'author': 'Explainee', 'text': 'virtual reality [SEP] That's one of the things we're trying to figure out, from an artist and storytelling perspective, what are the things that will make this meaningfully different from what we're used to, like a television on our wall. And we've been finding a lot of things, aspects of virtual reality that very much do that in my opinion. Things that allow you to feel presence, first and foremost, where you get lost, and you have to remind yourself, this isn't actually happening. And things that ultimately allow you to embody other characters. Things where you can actually change your own self-perception and play with neuro-plasticity and teach yourself things that are bizarre and unique.'}",5lvls,NaN,NaN,NaN,NaN
2,142,3,(T03) A related topic,(D10) To provide informing statement,(E03) Provide an explanation,virtual_reality,colleague,"{'author': 'Explainer', 'text': 'As an engineer of course, I love quantifiable things. I like saying here's my 18 millisecond motion to photon, here's my angular resolution that I'm improving. I'm doing the color-space right. But you can look not too far back where you say we have blu-ray DVDs at this amazing resolution, but more people want to watch Youtube videos at really bad early internet video speeds. Where there are things that if you deliver a value to people then these objective quantities may not be the most important thing. And while we're certainly pushing as hard as we can on lots of these things that make the experience better in potentially every way or maybe just for videos or for different things. I don't think that its necessary. I've commented that I think usually my favorite titles on mobile that are fully synthetic are ones that don't even try, they just go and do light-mapped, black-shaded. And I think its a lovely aesthetic. I think that you don't wind up fighting all of the aliasing. While you get some other titles that, oh we're gonna be high-tech with our specular bump maps with roughness. And you've got aliasing everywhere, and you can't hold frame rate and its all problematic. While some of these that are clearly very synthetic worlds where its nothing but these cartoony flat-shaded things with lighting, but they look and they feel good. And you can buy

### Experiments:

- Train three models, one on eli5, one on 5lvls, and one on both, and evaluate them in three settings, in-domain and out-domain - through a 5-fold cross validation setting

#### Prepare Folds:

In [13]:
print(len(dlgs_df[dlgs_df.ds == '5lvls'].topic.value_counts()))

13


In [14]:
dlgs_df[dlgs_df.ds == '5lvls'].topic.value_counts()

dimensions          244
gravity             179
music_harmony       150
sleep_scientist     147
lasers              142
machine_learning    119
origani             112
hacking             100
nano_technology      92
blockchain           91
blackhole            70
connectome           60
virtual_reality      44
Name: topic, dtype: int64

In [15]:
print(len(dlgs_df[dlgs_df.ds == 'eli5'].topic.value_counts()))

154


In [16]:
dlgs_df[dlgs_df.ds == 'eli5'].topic.value_counts()

How can some "infinites" be bigger than others? Doesn't that contradict the very definition of "infinite"?                                    153
Halting Problem and the Self-Referential Proof                                                                                                109
How does a computer learn those first few steps of computing?                                                                                 101
What's the point of buying non-dividend shares?                                                                                                78
Where did influenza go in 2020?                                                                                                                73
                                                                                                                                             ... 
How did the ideology of Karl Marx, that supported an utopian society where everyone would be happy, turned into Stalin or Ki

In [17]:
from sklearn.model_selection import KFold

In [18]:
#split the two corpora
eli5_topics  = dlgs_df[dlgs_df.ds == 'eli5'].topic.unique()
flvls_topics = dlgs_df[dlgs_df.ds == '5lvls'].topic.unique()

kfold = KFold(n_splits=5)
flvls_folds = [(flvls_topics[idx[0]], flvls_topics[idx[1]]) for idx in kfold.split(flvls_topics)]
eli5_folds  = [(eli5_topics[idx[0]], eli5_topics[idx[1]]) for idx in kfold.split(eli5_topics)]

eli5_training_folds, eli5_test_folds = zip(*eli5_folds)
flvls_training_folds, flvls_test_folds = zip(*flvls_folds)

In [20]:
all_folds = {'train':{'5lvls': [x.tolist() for x in flvls_training_folds], 'eli5': [x.tolist() for x in eli5_training_folds]},
             'test':{'5lvls': [x.tolist() for x in flvls_test_folds], 'eli5': [x.tolist() for x in eli5_test_folds]}}

In [21]:
json.dump(all_folds, open('../data/topic_folds.json', 'w'))

### Training models:

Now we train models via command line running turn_label_prediction_experiment_with_bert

The code for training the models is in: turn_label_prediction_experiments_with_bert and with_bert_seq

### Prediction using the trained models:

In [9]:
def majority_class(df):
    topics = df.topic.unique()
    for topic in topics:
        training_df = df[df.topic != topic]
        #compute the majority class for each label
        l = len(df[df.topic == topic])
        df.loc[df.topic == topic, 'topic_func_maj_pred'] = [training_df.topic_func_label.mode()] * l
        df.loc[df.topic == topic, 'dlg_act_maj_pred']    = [training_df.dlg_act_label.mode()] * l
        df.loc[df.topic == topic, 'exp_act_maj_pred']    = [training_df.exp_act_label.mode()] * l
    
    return df

def eval_preds(df, models_names, gt_clms, pred_clms):
    results_table = []
    for label in zip(gt_clms, pred_clms, models_names):
        ground_truths = df[label[0]].tolist()
        predictions   = df[label[1]].tolist()
        model_name = label[2]
        
        class_names = df[label[0]].unique()

        prc_scores = precision_score(ground_truths, predictions, average=None, labels=class_names)
        rec_scores = recall_score(ground_truths, predictions, average=None, labels=class_names)
        f1_scores  = f1_score(ground_truths, predictions, average=None, labels=class_names)
        
        macro_prc_scores = precision_score(predictions, ground_truths, average='macro', labels=class_names)
        macro_rec_scores = recall_score(predictions, ground_truths, average='macro', labels=class_names)
        macro_f1 = f1_score(predictions, ground_truths, average='macro', labels=class_names)
        
        scores ={}
        for i, c in enumerate(class_names):
            scores[c] = {'prec': round(prc_scores[i],2), 'recall': round(rec_scores[i],2), 'f1': round(f1_scores[i],2)}
        
        scores['Macro AVG.'] = {'prec': round(macro_prc_scores,2), 'recall': round(macro_rec_scores,2), 'f1': round(macro_f1,2)}
        
        results_table.append([model_name, label[0], scores])
    
    return results_table

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(-1)
    f1score = f1_score(predictions, labels, average='macro')
    return {'f1-score': f1score}

def evaluate_fold(fold_path, df, fold_idx, input_clm, tokenizer, label_clm='exp_act_label', ds_names=[]):
    
    model = AutoModelForSequenceClassification.from_pretrained(fold_path + 'best_model').to(device)
    model.eval()
    fold_topics = []
    if 'eli5' in ds_names:
        fold_topics += folds_dict['test']['eli5'][fold_idx]
    if '5lvls' in ds_names:
        fold_topics += folds_dict['test']['5lvls'][fold_idx]
    
    test_df  = df[df.topic.isin(fold_topics)]
    test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1) 
    
    eval_dataset = Dataset.from_pandas(test_df)
    eval_dataset = eval_dataset.map(lambda examples: tokenizer([x['text'] for x in examples[input_clm]], truncation=True, padding='max_length'), batched=True)
    eval_dataset = eval_dataset.remove_columns(test_df.columns.tolist() + ['__index_level_0__'])
    
    
    eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator)
    all_predictions = []
    for step, batch in enumerate(eval_dataloader):
        batch = {x[0]: x[1].cuda() for x in batch.items()}
        outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        all_predictions+=[x.item() for x in predictions]

    #print(all_predictions)
    #print(test_df['labels'].tolist())
    
    return f1_score(all_predictions, test_df['labels'].tolist(), average='macro')

def evaluate_model(models_path, label_clm, label_model, model_name='bert-base-uncased'):
    f1_scores = {}

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    for model_name in ['eli5_models', '5lvls_models', 'all_models']:
        model_scores = []
        for ds_names in [['eli5'], ['5lvls'], ['5lvls', 'eli5']]:
            s = np.mean([evaluate_fold('{}/{}/{}/model/fold-{}/'.format(models_path, label_model, model_name, fold), dlgs_df.copy(), 
                                       fold, 'turn_text_with_topic', tokenizer, label_clm, ds_names=ds_names) 
                         for fold in range(5)])
            model_scores.append(round(s, 2))
        f1_scores[model_name] = model_scores
    
    return f1_scores

In [10]:
folds_dict = json.load(open('../data/topic_folds.json'))

In [11]:
dlg_act_f1_scores = evaluate_model('../data/turn-label-models/bert-base-uncased/', 'dlg_act_label', 'dlg_act_label_prediction')

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)
Parameter 'function'=<function evaluate_fold.<locals>.<lambda> at 0x7f065c33ac10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
exp_act_f1_scores = evaluate_model('../data/turn-label-models/bert-base-uncased/', 'exp_act_label', 'exp_act_label_prediction')

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/2 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
topic_func_f1_scores = evaluate_model('../data/turn-label-models/bert-base-uncased/', 'topic_func_label', 'topic_func_label_prediction')

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/2 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
print(tabulate([[x[0][0]] + x[0][1] + x[1][1] + x[2][1] for x in zip(exp_act_f1_scores.items(), dlg_act_f1_scores.items(), topic_func_f1_scores.items())], 
               headers=['Approach', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL']))

Approach        ELI5    5lvls    ALL    ELI5    5lvls    ALL    ELI5    5lvls    ALL
------------  ------  -------  -----  ------  -------  -----  ------  -------  -----
eli5_models     0.33     0.26   0.29    0.34     0.28   0.35    0.37     0.4    0.43
5lvls_models    0.17     0.38   0.32    0.14     0.46   0.31    0.2      0.46   0.38
all_models      0.3      0.37   0.39    0.4      0.44   0.48    0.38     0.49   0.48


#### Evaluate on Roberta:

In [15]:
roberta_dlg_act_f1_scores = evaluate_model('../data/turn-label-models/roberta-base/', 'dlg_act_label', 'dlg_act_label_prediction', model_name='roberta-base')

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/2 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
roberta_exp_act_f1_scores = evaluate_model('../data/turn-label-models/roberta-base/', 'exp_act_label', 'exp_act_label_prediction', model_name='roberta-base')

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/2 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [17]:
roberta_topic_func_f1_scores = evaluate_model('../data/turn-label-models/roberta-base/', 'topic_func_label', 'topic_func_label_prediction', model_name='roberta-large')

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/2 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


...

  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

<ipython-input-9-47702c4f37b4>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['labels'] = test_df[label_clm].apply(lambda x: int(x[2:4])-1)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
print(tabulate([[x[0][0]] + x[0][1] + x[1][1] + x[2][1] for x in zip(roberta_exp_act_f1_scores.items(), roberta_dlg_act_f1_scores.items(), roberta_topic_func_f1_scores.items())], 
               headers=['Approach', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL', 'ELI5', '5lvls', 'ALL']))

Approach        ELI5    5lvls    ALL    ELI5    5lvls    ALL    ELI5    5lvls    ALL
------------  ------  -------  -----  ------  -------  -----  ------  -------  -----
eli5_models     0.36     0.22   0.27    0.39     0.22   0.38    0.39     0.39   0.43
5lvls_models    0.15     0.35   0.28    0.15     0.34   0.27    0.28     0.54   0.43
all_models      0.32     0.34   0.36    0.41     0.34   0.44    0.39     0.55   0.49


--------

### Predicting on the test set:

- Best Performing Models:
    - Explanation Moves: ELI-5 trained BERT-Seq
    - Dialogue Acts: Both trained RoBERTa
    - Topic Func: ELI-5 trained RoBERTa

In [40]:
test_df = load_ds('../../data/eli5_ds/annotation-results/MACE-measure/final_mace_predictions_testing.pkl')

In [42]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [43]:
def ensemble_generate(model_path, test_df, input_clm, tokenizer, label_clm='exp_act_label'):
    label_dictionary = {int(l[2:4])-1 : l for l in  test_df[label_clm].unique()}
    print(label_dictionary)
    
    models = []
    for fold_path in glob.glob(model_path):
        model = AutoModelForSequenceClassification.from_pretrained(fold_path + '/best_model').to(device)
        model.eval()
        models.append(model)
        
    eval_dataset = Dataset.from_pandas(test_df)
    eval_dataset = eval_dataset.map(lambda examples: tokenizer([x['text'] for x in examples[input_clm]], truncation=True, padding='max_length'), batched=True)
    eval_dataset = eval_dataset.remove_columns(test_df.columns.tolist() + ['__index_level_0__'])
    
    
    eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=4)
    all_predictions = []
    for step, batch in enumerate(eval_dataloader):
        batch = {x[0]: x[1].cuda() for x in batch.items()}
        models_outputs = [model(**batch) for model in models]
        models_predictions = [outputs.logits.argmax(dim=-1) for outputs in models_outputs]
        models_predictions =[[x.item() for x in predictions] for predictions in models_predictions]
        models_predictions = [max(set(lst), key=lst.count) for lst in list(zip(*models_predictions))]
        all_predictions += models_predictions
        #print(batch['input_ids'].shape)
        #print(models_outputs)
        #print(models_predictions)

    test_df[label_clm + '_predictions'] = [label_dictionary[x] for x in all_predictions]
    return test_df

In [45]:
test_df = ensemble_generate('../data/roberta/dlg_act_prediction/mixed_ds_models/model/fold*', test_df, 'turn_text_with_topic', tokenizer, label_clm='dlg_act_label')

{1: '(D02) To ask what/how question', 2: '(D03) To ask other kind of questions', 5: '(D06) Answer - Other', 7: '(D08) Disagreement', 4: '(D05) To answer a question by disconfirming', 9: '(D10) To provide informing statement', 3: '(D04) To answer a question by confirming', 0: '(D01) To ask a check question', 6: '(D07) Agreement', 8: '(D09) Other'}


  0%|          | 0/2 [00:00<?, ?ba/s]

In [47]:
print('Prec. {} Rec. {}, F1-score {}'.format(
    precision_score(test_df.dlg_act_label_predictions, test_df.dlg_act_label, average='macro'),
    recall_score(test_df.dlg_act_label_predictions, test_df.dlg_act_label, average='macro'),
    f1_score(test_df.dlg_act_label_predictions, test_df.dlg_act_label, average='macro')
))

Prec. 0.40944300490612545 Rec. 0.39383271789297913, F1-score 0.38164156899072377


In [48]:
test_df = ensemble_generate('../data/roberta/topic_func_prediction/eli5_ds_models/model/fold*', test_df, 'turn_text_with_topic', tokenizer, label_clm='topic_func_label')

{0: '(T01) It is the main topic', 1: '(T02) A subtopic', 3: '(T04) Other - No topic was introduced', 2: '(T03) A related topic'}


  0%|          | 0/2 [00:00<?, ?ba/s]

In [49]:
print('Prec. {} Rec. {}, F1-score {}'.format(
    precision_score(test_df.topic_func_label_predictions, test_df.topic_func_label, average='macro'),
    recall_score(test_df.topic_func_label_predictions, test_df.topic_func_label, average='macro'),
    f1_score(test_df.topic_func_label_predictions, test_df.topic_func_label, average='macro')
))

Prec. 0.48531420535051 Rec. 0.46274504909244196, F1-score 0.46322998185265796


In [60]:
test_df.head()

,task_id,turn_id,topic_func_label,dlg_act_label,exp_act_label,exp_type_label,post_link,post_id,turn_text,topic,turn_text_only,turn_text_with_topic,exp_act_label_predictions
40,28223,1,(T01) It is the main topic,(D02) To ask what/how question,(E04) Ask for an explanation,NaN,https://www.reddit.com/r/explainlikeimfive/comments/dlzify/eli5_how_is_it_possible_that_there_can_be_a/,dlzify,"{'author': 'Explainee', 'text': 'How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon?'}",How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon?,How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon?,"{'author': 'Explainee', 'text': 'How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon? [SEP] How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon?'}",(E02) Testing prior knowledge
41,28223,2,(T01) It is the main topic,(D02) To ask what/how question,(E03) Provide an explanation,(Y02) Contextual,https://www.reddit.com/r/explainlikeimfive/comments/dlzify/eli5_how_is_it_possible_that_there_can_be_a/,dlzify,"{'author': 'Explainer', 'text': '&gt;Thats not what I'm asking. But it is the answer. &gt;In the video the moon is setting into the earths umbra No it's not. The moon setting isn't the Moon moving, it's the Earth rotating and does nothing to move the umbra. &gt;yet the shadow appears from the top down. How is that possible? The Moon's orbit places it in the Earth's umbra. This happens in exactly the same way no matter where the observer is (ignoring relativistic effects).'}",How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon?,"&gt;Thats not what I'm asking. \n\nBut it is the answer. \n\n&gt;In the video the moon is setting into the earths umbra\n\nNo it's not. The moon setting isn't the Moon moving, it's the Earth rotating and does nothing to move the umbra. \n\n&gt;yet the shadow appears from the top down. How is that possible?\n\nThe Moon's orbit places it in the Earth's umbra. This happens in exactly the same way no matter where the observer is (ignoring relativistic effects).","{'author': 'Explainer', 'text': 'How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon? [SEP] &gt;Thats not what I'm asking. But it is the answer. &gt;In the video the moon is setting into the earths umbra No it's not. The moon setting isn't the Moon moving, it's the Earth rotating and does nothing to move the umbra. &gt;yet the shadow appears from the top down. How is that possible? The Moon's orbit places it in the Earth's umbra. This happens in exactly the same way no matter where the observer is (ignoring relativistic effects).'}",(E05) Signaling understanding
42,28223,3,(T01) It is the main topic,(D03) To ask other kind of questions,(E04) Ask for an explanation,NaN,https://www.reddit.com/r/explainlikeimfive/comments/dlzify/eli5_how_is_it_possible_that_there_can_be_a/,dlzify,"{'author': 'Explainee', 'text': 'So if the umbra is moving *towards* the moon then why does the shadow appear from the top down? Should it not appear bottom first and make its way up?'}",How is it possible that there can be a selenelion eclipse with the moon setting into the earths umbra and a shadow appearing on the topside of the moon?,So if the umbra is moving *towards* the moon then why does the shadow appear from the top down? Should it not appear bottom first and make its way up?,"{'author': 'Explainee', 'text': '

In [50]:
test_df.to_pickle('../data/final_mace_predictions_testing_with_predictions.pkl')